In [1]:
import numpy as np
import collections.abc
collections.Iterable = collections.abc.Iterable
from collections import OrderedDict
import pylogit as pl
import pandas as pd
import scipy.special
import json

In [2]:
df = pd.read_csv('history.csv')
df

,round,arena,pirate1,pirate2,pirate3,pirate4,fa1,fa2,fa3,fa4,...,nfa4,opening_odds1,opening_odds2,opening_odds3,opening_odds4,closing_odds1,closing_odds2,closing_odds3,closing_odds4,winner
0,3574,0,7,19,4,13,0,0,0,0,...,0,5,2,3,9,6,2,3,11,3
1,3574,1,18,12,17,14,0,0,0,0,...,0,13,2,3,3,13,2,3,3,4
2,3574,2,2,16,9,3,0,0,0,0,...,0,13,2,5,4,13,2,6,5,2
3,3574,3,10,20,1,6,0,0,0,0,...,0,13,10,2,13,13,13,2,13,3
4,3574,4,8,15,11,5,0,0,0,0,...,0,13,2,11,6,13,2,13,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16840,8641,0,19,12,2,15,2,4,3,3,...,-1,4,13,13,2,4,13,13,2,4
16841,8641,1,10,20,8,4,1,-1,-2,3,...,0,13,8,13,2,13,12,13,2,4
16842,8641,2,17,7,18,6,4,-1,2,1,...,-1,2,6,5,4,2,6,5,4,1
16843,8641,3,16,1,5,14,-2,-1,4,1,...,-1,13,4,2,5,13,5,2,7,2


In [3]:
df.describe()

,round,arena,pirate1,pirate2,pirate3,pirate4,fa1,fa2,fa3,fa4,...,nfa4,opening_odds1,opening_odds2,opening_odds3,opening_odds4,closing_odds1,closing_odds2,closing_odds3,closing_odds4,winner
count,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,...,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000
mean,6829.771149,2.000000,10.591036,10.431345,10.531374,10.446245,0.970377,0.985634,0.964797,0.985159,...,-1.208549,7.915702,7.034194,5.982191,4.689225,8.237934,7.407183,6.360641,5.058949,2.744672
std,1089.898648,1.414256,5.751749,5.783485,5.783417,5.745589,1.731506,1.735176,1.731504,1.731130,...,0.970179,4.460321,4.417173,4.182325,3.636474,4.533579,4.551443,4.382981,3.927148,1.106017
min,3574.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-5.000000,-5.000000,-5.000000,-5.000000,...,-6.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000
25%,5932.000000,1.000000,6.000000,5.000000,6.000000,6.000000,0.000000,0.000000,0.000000,0.000000,...,-2.000000,3.000000,3.000000,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000,2.000000
50%,6857.000000,2.000000,11.000000,10.000000,11.000000,10.000000,1.000000,1.000000,1.000000,1.000000,...,-1.000000,7.000000,6.000000,4.000000,3.000000,9.000000,7.000000,5.000000,3.000000,3.000000
75%,7799.000000,3.000000,16.000000,15.000000,16.000000,15.000000,2.000000,2.000000,2.000000,2.000000,...,0.000000,13.000000,13.000000,10.000000,6.000000,13.000000,13.000000,11.000000,7.000000,4.000000
max,8641.000000,4.000000,20.000000,20.000000,20.000000,20.000000,8.000000,8.000000,8.000000,9.000000,...,0.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,4.000000


In [4]:
# combine columns pirate1 through pirate4 into a single column containing a list of pirate choices
def convert_df(df_orig):
    df = df_orig.copy()
    df['pirates'] = df[['pirate1', 'pirate2', 'pirate3', 'pirate4']].values.tolist()
    df['fas'] = df[['fa1', 'fa2', 'fa3', 'fa4']].values.tolist()
    df['pfas'] = df[['pfa1', 'pfa2', 'pfa3', 'pfa4']].values.tolist()
    df['nfas'] = df[['nfa1', 'nfa2', 'nfa3', 'nfa4']].values.tolist()
    df['opening_odds'] = df[['opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4']].values.tolist()
    df['closing_odds'] = df[['closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']].values.tolist()
    df = df.drop(['pirate1', 'pirate2', 'pirate3', 'pirate4'], axis=1)
    df = df.drop(['fa1', 'fa2', 'fa3', 'fa4'], axis=1)
    df = df.drop(['pfa1', 'pfa2', 'pfa3', 'pfa4'], axis=1)
    df = df.drop(['nfa1', 'nfa2', 'nfa3', 'nfa4'], axis=1)
    df = df.drop(['opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4'], axis=1)
    df = df.drop(['closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4'], axis=1)
    df['match_id'] = df['round'] * 5 + df['arena']
    return df
df = convert_df(df)
df

,round,arena,winner,pirates,fas,pfas,nfas,opening_odds,closing_odds,match_id
0,3574,0,3,"[7, 19, 4, 13]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[5, 2, 3, 9]","[6, 2, 3, 11]",17870
1,3574,1,4,"[18, 12, 17, 14]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 3, 3]","[13, 2, 3, 3]",17871
2,3574,2,2,"[2, 16, 9, 3]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 5, 4]","[13, 2, 6, 5]",17872
3,3574,3,3,"[10, 20, 1, 6]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 10, 2, 13]","[13, 13, 2, 13]",17873
4,3574,4,2,"[8, 15, 11, 5]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 11, 6]","[13, 2, 13, 6]",17874
...,...,...,...,...,...,...,...,...,...,...
16840,8641,0,4,"[19, 12, 2, 15]","[2, 4, 3, 3]","[2, 5, 5, 4]","[0, -1, -2, -1]","[4, 13, 13, 2]","[4, 13, 13, 2]",43205
16841,8641,1,4,"[10, 20, 8, 4]","[1, -1, -2, 3]","[1, 1, 1, 3]","[0, -2, -3, 0]","[13, 8, 13, 2]","[13, 12, 13, 2]",43206
16842,8641,2,1,"[17, 7, 18, 6]","[4, -1, 2, 1]","[5, 1, 4, 2]","[-1, -2, -2, -1]","[2, 6, 5, 4]","[2, 6, 5, 4]",43207
16843,8641,3,2,"[16, 1, 5, 14]","[-2, -1, 4, 1]","[0, 1, 4, 2]","[-2, -2, 0, -1]","[13, 4, 2, 5]","[13, 5, 2, 7]",43208


In [5]:
# explode each row of df into four rows, so that the resulting df
# contains columns of round, arena, win (0 or 1), pirate, fa, opening_odds, closing_odds, match_id
def convert_to_long_format_with_win(df):
    long_format_columns = ['round', 'arena', 'win', 'pirate', 'fa', 'pfa', 'nfa', 'position', 'position_factor', 'is_pos1', 'is_pos2', 'is_pos3', 'is_pos4', 'opening_odds', 'closing_odds', 'match_id']
    # positional effect is significant but sorta non-linear; this is taken from a model data
    # and we'll calculate each pirates "suscetibility to positional effects" by fitting a multiplier
    position_factor = [-0.1821, 0, 0.2814, 0.5523]
    long_format = pd.DataFrame([
        (
            tup.round,
            tup.arena,
            1 if 'winner' in df.columns and tup.winner == (i + 1) else 0,
            tup.pirates[i],
            tup.fas[i],
            tup.pfas[i],
            tup.nfas[i],
            i + 1,
            position_factor[i],
            1 if i + 1 == 1 else 0,
            1 if i + 1 == 2 else 0,
            1 if i + 1 == 3 else 0,
            1 if i + 1 == 4 else 0,
            tup.opening_odds[i],
            tup.closing_odds[i],
            tup.match_id
        )
        for tup in df.itertuples() for i in range(4)
    ],columns=long_format_columns)
    long_format['log_opening_implied_winrate'] = np.log(1.0 / long_format['opening_odds'])
    return long_format

def convert_to_long_format_without_win(df):
    return convert_to_long_format_with_win(df).drop(['win'], axis=1)

long_format = convert_to_long_format_with_win(df)
long_format

,round,arena,win,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,3574,0,0,7,0,0,0,1,-0.1821,1,0,0,0,5,6,17870,-1.609438
1,3574,0,0,19,0,0,0,2,0.0000,0,1,0,0,2,2,17870,-0.693147
2,3574,0,1,4,0,0,0,3,0.2814,0,0,1,0,3,3,17870,-1.098612
3,3574,0,0,13,0,0,0,4,0.5523,0,0,0,1,9,11,17870,-2.197225
4,3574,1,0,18,0,0,0,1,-0.1821,1,0,0,0,13,13,17871,-2.564949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67375,8641,3,0,14,1,2,-1,4,0.5523,0,0,0,1,5,7,43208,-1.609438
67376,8641,4,0,11,0,1,-1,1,-0.1821,1,0,0,0,11,8,43209,-2.397895
67377,8641,4,1,13,-1,1,-2,2,0.0000,0,1,0,0,3,4,43209,-1.098612
67378,8641,4,0,3,3,4,-1,3,0.2814,0,0,1,0,3,3,43209,-1.098612


In [6]:
PIRATE_NAMES = {
    1: "Dan",
    2: "Sproggie",
    3: "Orvinn",
    4: "Lucky",
    5: "Edmund",
    6: "Peg Leg",
    7: "Bonnie",
    8: "Puffo",
    9: "Stuff",
    10: "Squire",
    11: "Crossblades",
    12: "Stripey",
    13: "Ned",
    14: "Fairfax",
    15: "Gooblah",
    16: "Franchisco",
    17: "Federismo",
    18: "Blackbeard",
    19: "Buck",
    20: "Tailhook",
}

In [7]:
pirates_including_the_first = list(range(1, 21))
pirates_except_the_first = list(range(2, 21))

In [8]:
POSITIVE_FAS = {
    1: {1: 2, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 2, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 2, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    2: {1: 1, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0},
    3: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 1, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 1},
    4: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 1},
    5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    6: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    7: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    8: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 1, 38: 1, 39: 0, 40: 0},
    9: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 1},
    10: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    11: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 1},
    12: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0},
    13: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    14: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    15: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    16: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 2, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    17: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 2, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 2},
    18: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    19: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    20: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0}
}
NEGATIVE_FAS = {
    1: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    2: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 1},
    3: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    4: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 1},
    5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    6: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    7: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    8: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    9: {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0},
    10: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    11: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    12: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    13: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    14: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    15: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0},
    16: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    17: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    18: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    19: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    20: {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0}
}

def get_ongoing_df_from_file(path):
    columns = ['round', 'arena', 'pirate1', 'pirate2', 'pirate3', 'pirate4', 'fa1', 'fa2', 'fa3', 'fa4', 'opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4', 'closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']
    df = pd.DataFrame([], columns=columns)
    with open(path) as json_file:
        data = json.load(json_file)
        round = data["round"]
        for arena in range(5):
            pirates = data["pirates"][arena]
            # older rounds don't have fa data
            fas = [0 for pirate in pirates]
            if "foods" in data:
                foods = data["foods"][arena]
                positive_fas = [sum(POSITIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                negative_fas = [-sum(NEGATIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                fas = [sum(x) for x in zip(positive_fas, negative_fas)]
            pirate1, pirate2, pirate3, pirate4 = pirates
            fa1, fa2, fa3, fa4 = fas
            _, opening_odds1, opening_odds2, opening_odds3, opening_odds4 = data["openingOdds"][arena]
            _, closing_odds1, closing_odds2, closing_odds3, closing_odds4 = data["currentOdds"][arena]
            df_temp = pd.DataFrame([[round, arena, pirate1, pirate2, pirate3, pirate4, fa1, fa2, fa3, fa4, opening_odds1, opening_odds2, opening_odds3, opening_odds4, closing_odds1, closing_odds2, closing_odds3, closing_odds4]], columns=columns)
            df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [9]:
# backtesting
def get_data_for_round(df, round):
    return df[df['round'] == round]

convert_to_long_format_with_win(get_data_for_round(df, 8641))


,round,arena,win,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,8641,0,0,19,2,2,0,1,-0.1821,1,0,0,0,4,4,43205,-1.386294
1,8641,0,0,12,4,5,-1,2,0.0000,0,1,0,0,13,13,43205,-2.564949
2,8641,0,0,2,3,5,-2,3,0.2814,0,0,1,0,13,13,43205,-2.564949
3,8641,0,1,15,3,4,-1,4,0.5523,0,0,0,1,2,2,43205,-0.693147
4,8641,1,0,10,1,1,0,1,-0.1821,1,0,0,0,13,13,43206,-2.564949
5,8641,1,0,20,-1,1,-2,2,0.0000,0,1,0,0,8,12,43206,-2.079442
6,8641,1,0,8,-2,1,-3,3,0.2814,0,0,1,0,13,13,43206,-2.564949
7,8641,1,1,4,3,3,0,4,0.5523,0,0,0,1,2,2,43206,-0.693147
8,8641,2,1,17,4,5,-1,1,-0.1821,1,0,0,0,2,2,43207,-0.693147
9,8641,2,0,7,-1,1,-2,2,0.0000,0,1,0,0,6,6,43207,-1.791759


In [10]:
def generate_bets():
    for i in range(5):
        for j in range(5):
            for k in range(5):
                for l in range(5):
                    for m in range(5):
                        if i != 0 or j != 0 or k != 0 or l != 0 or m != 0:
                            yield (i, j, k, l, m)

len(list(generate_bets()))

3124

In [11]:
def get_odds_of_bet(bet, odds):
    prod = 1
    for arena, pos in enumerate(bet):
        if pos != 0:
            prod *= odds[arena * 4 + pos - 1]
    return prod

tmp = convert_to_long_format_with_win(get_data_for_round(df, 8641))
get_odds_of_bet((4,2,0,0,2), tmp['closing_odds'])

96

In [12]:
def get_estimated_probabilities(df, model):
    return model.predict(df)

# get_estimated_probabilities(tmp, with_position_mnl)

In [13]:
def get_estimated_probability_of_bet(bet, probabilities):
    prod = 1
    for arena, pos in enumerate(bet):
        if pos != 0:
            prod *= probabilities[arena * 4 + pos - 1]
    return prod

# get_estimated_probability_of_bet((4,2,0,0,2), get_estimated_probabilities(tmp, with_position_mnl))

In [14]:
def get_estimated_return_per_unit_of_bet(bet, odds, probabilities):
    return get_odds_of_bet(bet, odds) * get_estimated_probability_of_bet(bet, probabilities)

# get_estimated_return_per_unit_of_bet((4,2,0,0,2), tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl))

In [15]:
def get_max_possible_bet_amount_floor(odds):
    return 1000000 // odds

def get_max_possible_bet_amount_ceil(odds):
    return -(1000000 // -odds)

get_max_possible_bet_amount_floor(96)
get_max_possible_bet_amount_ceil(96)

10417

In [16]:
def get_estimated_return_of_bet_amount(bet, odds, probabilities, bet_amount):
    odds_of_bet = get_odds_of_bet(bet, odds)
    estimated_probability = get_estimated_probability_of_bet(bet, probabilities)
    payout = min(1000000, bet_amount * odds_of_bet)
    return payout * estimated_probability

def get_best_bet_amount_and_estimated_return(bet, odds, probabilities):
    max_possible_bet_amount_floor = get_max_possible_bet_amount_floor(get_odds_of_bet(bet, odds))
    max_possible_bet_amount_ceil = get_max_possible_bet_amount_ceil(get_odds_of_bet(bet, odds))
    estimated_return_bet_amount_floor = get_estimated_return_of_bet_amount(bet, odds, probabilities, max_possible_bet_amount_floor)
    estimated_return_bet_amount_ceil = get_estimated_return_of_bet_amount(bet, odds, probabilities, max_possible_bet_amount_ceil)
    if (estimated_return_bet_amount_floor - max_possible_bet_amount_floor) >= (estimated_return_bet_amount_ceil - max_possible_bet_amount_ceil):
        return max_possible_bet_amount_floor, estimated_return_bet_amount_floor
    else:
        return max_possible_bet_amount_ceil, estimated_return_bet_amount_ceil

def get_best_bet_amount_and_estimated_return_given_personal_cap(bet, odds, probabilities, cap):
    amount, estimated_return = get_best_bet_amount_and_estimated_return(bet, odds, probabilities)
    if cap >= amount:
        return amount, estimated_return
    else:
        return cap, get_estimated_return_of_bet_amount(bet, odds, probabilities, cap)


In [17]:
def get_profitable_bets(odds, probabilities, cap):
    for bet in generate_bets():
        if get_estimated_return_per_unit_of_bet(bet, odds, probabilities) > 1:
            bet_amount, estimated_return = get_best_bet_amount_and_estimated_return_given_personal_cap(bet, odds, probabilities, cap)
            estimated_profit = estimated_return - bet_amount
            return_if_won = min(1000000, bet_amount * get_odds_of_bet(bet, odds))
            yield bet, bet_amount, estimated_return, estimated_profit, return_if_won

# 10 best bets for round 8641
# sorted(list(get_profitable_bets(tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl), 11376)), key=lambda x: x[3], reverse=True)[:10]

In [18]:
def is_winning_bet(bet, winners):
    for arena, pos in enumerate(bet):
        if pos != 0 and winners[arena] != pos:
            return False
    return True

def get_actual_return_of_bet(bet, winners, return_if_won):
    if is_winning_bet(bet, winners):
        return min(1000000, return_if_won)
    else:
        return 0


In [19]:
history_rounds = df["round"].unique().tolist()
len(history_rounds)

3369

In [20]:
from tqdm import tqdm
def backtest_model_with_cap(mnl, cap, history_rounds):
    total_bet_amount = 0
    total_return = 0
    total_potential_bet_amount = cap * 10 * len(history_rounds)
    for round in tqdm(history_rounds):
        tmp = convert_to_long_format_with_win(get_data_for_round(df, round))
        winners = df[df['round'] == round]['winner'].tolist()
        best_ten_bets = sorted(list(get_profitable_bets(tmp['closing_odds'], get_estimated_probabilities(tmp, mnl), cap)), key=lambda x: x[3], reverse=True)[:10]
        for bet, bet_amount, estimated_return, estimated_profit, return_if_won in best_ten_bets:
            total_bet_amount += bet_amount
            total_return += get_actual_return_of_bet(bet, winners, return_if_won)
    return total_bet_amount, total_potential_bet_amount, total_return, total_return - total_bet_amount, 1 + (total_return - total_bet_amount) / total_potential_bet_amount

In [21]:
# next we split positional coefficients for each pirate
per_pirate_fa_and_pos_specification = OrderedDict()
per_pirate_fa_and_pos_names = OrderedDict()
# OK, for asthetic purposes and better-looking P-values
# (Dan is really close to Buck but everyone is much weaker than Gooblah,
#  that's why Buck has a bad-looking P-value) we exlude Gooblah from the intercepts
# You can interpret each exp(intercept) as "how many Goobs each pirate is equivalent to"
pirates_except_goob = [i for i in range(1, 21) if i != 15]
per_pirate_fa_and_pos_specification["intercept"] = pirates_except_goob
per_pirate_fa_and_pos_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_goob]
per_pirate_fa_and_pos_specification["pfa"] = pirates_including_the_first
per_pirate_fa_and_pos_names["pfa"] = [f'PFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_pos_specification["nfa"] = pirates_including_the_first
per_pirate_fa_and_pos_names["nfa"] = [f'NFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_pos_specification["position_factor"] = pirates_including_the_first
per_pirate_fa_and_pos_names["position_factor"] = [f'position_factor_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]

per_pirate_fa_and_pos_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=per_pirate_fa_and_pos_specification,
model_type='MNL',
names=per_pirate_fa_and_pos_names)
per_pirate_fa_and_pos_mnl.fit_mle(np.zeros(79))
per_pirate_fa_and_pos_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 6.44 seconds.
Final log-likelihood: -18,078.1295


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,766
Method:                                MLE   Df Model:                           79
Date:                     Tue, 24 Jan 2023   Pseudo R-squ.:                   0.226
Time:                             00:37:04   Pseudo R-bar-squ.:               0.222
AIC:                            36,314.259   Log-Likelihood:            -18,078.129
BIC:                            36,925.072   LL-Null:                   -23,352.129
==================================================================================================
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
ASC_1_Dan                         -0.6198      0.130     -4.763      0.000      -0.875      -0.365
ASC_2_Sproggie                    -2.3731      0.175    -13.545      0.000      -2.717      -2.030
ASC_3_Orvinn                      -3.3557      0.212    -15.861      0.000      -3.770      -2.941
ASC_4_Lucky                       -1.2663      0.130     -9.723      0.000      -1.522      -1.011
ASC_5_Edmund                      -1.7855      0.133    -13.471      0.000      -2.045      -1.526
ASC_6_Peg Leg                     -2.4166      0.143    -16.944      0.000      -2.696      -2.137
ASC_7_Bonnie                      -2.1906      0.150    -14.649      0.000      -2.484      -1.897
ASC_8_Puffo                       -2.9591      0.210    -14.098      0.000      -3.370      -2.548
ASC_9_Stuff                       -3.8389      0.204    -18.819      0.000      -4.239      -3.439
ASC_10_Squire                     -3.2848      0.180    -18.290      0.000      -3.637      -2.933
ASC_11_Crossblades                -2.8585      0.175    -16.306      0.000      -3.202      -2.515
ASC_12_Stripey                    -2.1179      0.149    -14.168      0.000      -2.411      -1.825
ASC_13_Ned                        -1.5461      0.148    -10.437      0.000      -1.836      -1.256
ASC_14_Fairfax                    -2.5004      0.163    -15.332      0.000      -2.820      -2.181
ASC_16_Franchisco                 -1.3108      0.139     -9.451      0.000      -1.583      -1.039
ASC_17_Federismo                  -1.1296      0.136     -8.288      0.000      -1.397      -0.862
ASC_18_Blackbeard                 -2.0534      0.154    -13.327      0.000      -2.355      -1.751
ASC_19_Buck                       -0.5117      0.129     -3.961      0.000      -0.765      -0.259
ASC_20_Tailhook                   -1.5502      0.132    -11.724      0.000      -1.809      -1.291
PFA_1_Dan                          0.1701      0.023      7.519      0.000       0.126       0.214
PFA_2_Sproggie                     0.2553      0.035      7.239      0.000       0.186       0.324
PFA_3_Orvinn                       0.2560      0.041      6.215      0.000       0.175       0.337
PFA_4_Lucky                        0.1616      0.041      3.971      0.000       0.082       0.241
PFA_5_Edmund                       0.2680      0.039      6.842      0.000       0.191       0.345
PFA_6_Peg Leg                      0.3154      0.061      5.195      0.000       0.196       0.434
PFA_7_Bonnie                       0.2449      0.037      6.654      0.000       0.173       0.317
PFA_8_Puffo                        0.2885      0.040      7.285      0.000       0.211       0.366
PFA_9_Stuff                        0.3963      0.078      5.085      0.000       0.244       0.549
PFA_10_Squire                      0.2337      0.083      2.802      0.005       0.070       0.397
PFA_11_Crossblades          

In [22]:
def get_current_round_df_from_file(path):
    columns = ['round', 'arena', 'pirate1', 'pirate2', 'pirate3', 'pirate4', 'fa1', 'fa2', 'fa3', 'fa4', 'pfa1', 'pfa2', 'pfa3', 'pfa4', 'nfa1', 'nfa2', 'nfa3', 'nfa4', 'opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4', 'closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']
    df = pd.DataFrame([], columns=columns)
    with open(path) as json_file:
        data = json.load(json_file)
        round = data["round"]
        for arena in range(5):
            pirates = data["pirates"][arena]
            # older rounds don't have fa data
            fas = [0 for pirate in pirates]
            positive_fas = [0 for pirate in pirates]
            negative_fas = [0 for pirate in pirates]
            if "foods" in data:
                foods = data["foods"][arena]
                positive_fas = [sum(POSITIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                negative_fas = [-sum(NEGATIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                fas = [sum(x) for x in zip(positive_fas, negative_fas)]
            pirate1, pirate2, pirate3, pirate4 = pirates
            fa1, fa2, fa3, fa4 = fas
            pfa1, pfa2, pfa3, pfa4 = positive_fas
            nfa1, nfa2, nfa3, nfa4 = negative_fas
            _, opening_odds1, opening_odds2, opening_odds3, opening_odds4 = data["openingOdds"][arena]
            _, closing_odds1, closing_odds2, closing_odds3, closing_odds4 = data["currentOdds"][arena]
            df_temp = pd.DataFrame([[round, arena, pirate1, pirate2, pirate3, pirate4, fa1, fa2, fa3, fa4, pfa1, pfa2, pfa3, pfa4, nfa1, nfa2, nfa3, nfa4, opening_odds1, opening_odds2, opening_odds3, opening_odds4, closing_odds1, closing_odds2, closing_odds3, closing_odds4]], columns=columns)
            df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [23]:
def get_round(round_number):
    return convert_to_long_format_without_win(convert_df(get_current_round_df_from_file(f"raw_json/{round_number}.json")))
round_8643 = get_round(8643)
round_8643

,round,arena,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,8643,0,2,-1,2,-3,1,-0.1821,1,0,0,0,13,13,43215,-2.564949
1,8643,0,16,2,2,0,2,0.0000,0,1,0,0,2,2,43215,-0.693147
2,8643,0,3,0,1,-1,3,0.2814,0,0,1,0,13,13,43215,-2.564949
3,8643,0,5,2,2,0,4,0.5523,0,0,0,1,2,2,43215,-0.693147
4,8643,1,9,-1,0,-1,1,-0.1821,1,0,0,0,13,13,43216,-2.564949
5,8643,1,18,1,2,-1,2,0.0000,0,1,0,0,13,13,43216,-2.564949
6,8643,1,1,4,4,0,3,0.2814,0,0,1,0,2,2,43216,-0.693147
7,8643,1,8,-1,2,-3,4,0.5523,0,0,0,1,13,13,43216,-2.564949
8,8643,2,19,0,1,-1,1,-0.1821,1,0,0,0,2,2,43217,-0.693147
9,8643,2,13,2,3,-1,2,0.0000,0,1,0,0,2,2,43217,-0.693147


In [24]:
per_pirate_fa_and_pos_mnl.predict(round_8643)

array([0.05343339, 0.38704053, 0.04310275, 0.51642334, 0.0095265 ,
       0.07873558, 0.8718608 , 0.03987712, 0.50995024, 0.29752494,
       0.05561974, 0.13690508, 0.08281387, 0.3355578 , 0.40114837,
       0.18047996, 0.04304779, 0.86484364, 0.05053249, 0.04157608])

In [25]:
import string
def parse_bets(s):
    indices = [string.ascii_lowercase.find(c) for c in s]
    values = []
    for index in indices:
        values.append(index // 5)
        values.append(index % 5)

In [26]:
def get_best_10_bets(round, odds, model, cap):
    return sorted(list(get_profitable_bets(
        odds,
        get_estimated_probabilities(get_round(round), model), cap)),
        key=lambda x: x[3], reverse=True)[:10]

In [27]:
def get_bet_url(round, bets):
    bets_array = [v for bet in bets for v in bet[0]]
    # turn bets_array into chunks of 2 (pad 0 if necessary)
    if len(bets_array) % 2 == 1:
        bets_array.append(0)
    bets_chunks = [bets_array[i:i+2] for i in range(0, len(bets_array), 2)]
    # turn bets_chunks into a string
    bets_string = "".join([string.ascii_lowercase[5 * x + y] for x, y in bets_chunks])
    return f"/#round={round}&b={bets_string}"

In [28]:
from coloraide import Color, stop
def get_background_for_payout(payout):
    return Color.interpolate(['#50c17f', stop('transparent', np.exp(-1)), '#f76c6c'], space='srgb', premultiplied=True)(np.exp(-payout)).to_string()

get_background_for_payout(1.2)

'rgb(80 193 127 / 0.18127)'

In [29]:
def get_payouts(round, model, odds_override=None):
    round_df = get_round(round)
    estimated_probabilities = get_estimated_probabilities(round_df, model)
    estimated_probabilities_series = pd.Series(estimated_probabilities, name="estimated_probabilities")
    round_df = round_df.join(estimated_probabilities_series)
    if odds_override is not None:
        round_df["custom_odds"] = pd.Series(odds_override, name="custom_odds")
    else:
        round_df["custom_odds"] = round_df["closing_odds"]
    round_df["payouts"] = round_df.apply(lambda row: row["custom_odds"] * row["estimated_probabilities"], axis=1)
    return round_df.style.applymap(lambda x: f"background-color: {get_background_for_payout(x)}", subset=["payouts"])

In [45]:
current_round = 8662

In [46]:
manual_odds = get_round(current_round)['closing_odds'].tolist()
manual_odds

[5, 2, 13, 6, 13, 13, 6, 2, 7, 9, 3, 2, 9, 13, 12, 2, 13, 2, 4, 13]

In [47]:
per_pirate_fa_and_pos_mnl.predict(get_round(current_round))

array([0.21980817, 0.49087848, 0.05651856, 0.2327948 , 0.02592834,
       0.01810401, 0.29008944, 0.6658782 , 0.22457917, 0.15405244,
       0.34891764, 0.27245076, 0.14652173, 0.06300558, 0.09968258,
       0.69079012, 0.01794961, 0.70178594, 0.19652914, 0.08373532])

In [33]:
manual_odds = [2, 5, 13, 5, 3, 6, 7, 2, 13, 3, 2, 3, 13, 6, 6, 2, 7, 13, 13, 2]

In [48]:
get_payouts(current_round, per_pirate_fa_and_pos_mnl, odds_override=manual_odds)

,round,arena,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate,intercept,estimated_probabilities,custom_odds,payouts
0,8662,0,13,0,2,-2,1,-0.182100,1,0,0,0,4,5,43310,-1.386294,1.000000,0.219808,5,1.099041
1,8662,0,4,1,2,-1,2,0.000000,0,1,0,0,2,2,43310,-0.693147,1.000000,0.490878,2,0.981757
2,8662,0,10,-2,0,-2,3,0.281400,0,0,1,0,13,13,43310,-2.564949,1.000000,0.056519,13,0.734741
3,8662,0,8,1,3,-2,4,0.552300,0,0,0,1,5,6,43310,-1.609438,1.000000,0.232795,6,1.396769
4,8662,1,11,0,1,-1,1,-0.182100,1,0,0,0,13,13,43311,-2.564949,1.000000,0.025928,13,0.337068
5,8662,1,9,0,1,-1,2,0.000000,0,1,0,0,13,13,43311,-2.564949,1.000000,0.018104,13,0.235352
6,8662,1,16,2,3,-1,3,0.281400,0,0,1,0,4,6,43311,-1.386294,1.000000,0.290089,6,1.740537
7,8662,1,1,3,4,-1,4,0.552300,0,0,0,1,2,2,43311,-0.693147,1.000000,0.665878,2,1.331756
8,8662,2,2,2,4,-2,1,-0.182100,1,0,0,0,5,7,43312,-1.609438,1.000000,0.224579,7,1.572054
9,8662,2,12,1,3,-2,2,0.000000,0,1,0,0,6,9,43312,-1.791759,1.000000,0.154052,9,1.386472


In [49]:
get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 1)

[((4, 3, 1, 4, 2), 1, 7.4111562495219125, 6.4111562495219125, 1008),
 ((4, 3, 1, 1, 2), 1, 7.0738263324863935, 6.0738263324863935, 4536),
 ((4, 3, 2, 4, 2), 1, 6.536263307644561, 5.536263307644561, 1296),
 ((4, 3, 1, 3, 2), 1, 6.416679799849155, 5.416679799849155, 6048),
 ((4, 3, 2, 1, 2), 1, 6.2387554579844915, 5.2387554579844915, 5832),
 ((1, 3, 1, 4, 2), 1, 5.831432830760595, 4.831432830760595, 840),
 ((4, 3, 1, 4, 4), 1, 5.747822418048717, 4.747822418048717, 6552),
 ((4, 4, 1, 4, 2), 1, 5.670581402102679, 4.670581402102679, 336),
 ((4, 3, 2, 3, 2), 1, 5.659185600811434, 4.659185600811434, 7776),
 ((1, 3, 1, 1, 2), 1, 5.566006399746464, 4.566006399746464, 3780)]

In [50]:
get_bet_url(current_round, get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 1))

'/#round=8662&b=xjoqhxooqrxllqwxjyvwxnlqh'

In [53]:
get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 11418)

[((0, 4, 1, 4, 2), 11418, 46354.62877562134, 34936.62877562134, 639408),
 ((0, 3, 1, 0, 2), 11418, 43850.58350774211, 32432.583507742107, 959112),
 ((0, 3, 1, 4, 0), 11418, 43163.517779828966, 31745.517779828966, 959112),
 ((4, 4, 0, 4, 2), 11418, 41186.04778671502, 29768.04778671502, 548064),
 ((0, 4, 2, 4, 2), 11418, 40882.42765427107, 29464.427654271072, 822096),
 ((0, 3, 3, 4, 2), 11418, 40339.2811975044, 28921.281197504402, 822096),
 ((4, 3, 0, 0, 2), 11418, 38961.20572051752, 27543.205720517522, 822096),
 ((0, 3, 0, 4, 2), 11418, 38537.53916042297, 27119.539160422974, 274032),
 ((4, 3, 0, 4, 0), 11418, 38350.74841237213, 26932.74841237213, 822096),
 ((2, 4, 1, 4, 2), 8929, 35586.80780246373, 26657.80780246373, 1000000)]

In [54]:
get_bet_url(current_round, get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 11418))

'/#round=8662&b=ejkqcdjeuweokswxakpwxecvw'